# Final Exam Review - extra figures

In [ ]:
# R function for creating multivarite correlations.
createCorrelated <- function (n,rhos = 0.6,
                                mus=c(0,1),sigmas=c(1,0.5), type = 'norm',
                                sig = nchar(max(abs(mus)))+2, do.plot=F){
  ## Input rhos length should be length(mus)-1, if scalar it is recycled
  nVars <- length(mus)
  if (length(rhos)==1 || length(rhos)<(nVars-1)){
    rhos <- rep(rhos, ceiling((nVars-1)/length(rhos)))[1:(nVars-1)]
  } else if(length(rhos) > (nVars-1)) {
    rhos <- rhos[1:(nVars-1)]
  }
  if(length(type) < nVars){
    type <- rep(type,ceiling(nVars/length(type)))[1:nVars]
  } else if(length(type) > nVars){
    type = type[1:nVars]
  }
  matmult <- match.fun('%*%')

  theta <- acos(rhos)
  uX <- matrix(NA,nrow=n,ncol=nVars)
  for (m in 1:nVars){
    uX[,m] <- rnorm(n,mus[m], sigmas[m])
  }
  uXcnt <- scale(uX,center=TRUE,scale=FALSE)
  Id <- diag(n)
  Q <- qr.Q(qr(uXcnt[,1,drop=FALSE])) #Use first column as correlator
  P <- tcrossprod(Q)
  Orthogs <- apply(as.matrix(uXcnt[,-1]),MARGIN = 2,FUN = function(c) (Id-P)%*%c)
  Ymat <- cbind(uXcnt[,1],Orthogs)
  Ymat <- Ymat %*% diag(1/sqrt(colSums(Ymat^2)))
  output <- cbind(
    uX[,1],
    Ymat[,rep(1,length(theta))] *
      matrix(rep(1/tan(theta),nrow(Ymat)),ncol = length(theta),byrow=T) +
      Ymat[,-1]
  )
  # realign to origninal mus
  output <- sweep(x = output, MARGIN = 2, mus - apply(output,2,mean),FUN = `+`)
  # Apply type

  # apply signifs
  output <- signif(output,digits = sig)

  if(do.plot){
    pairs(output)
  }
  invisible(output)
}